In [1]:
import psycopg2

# Fonction pour établir une connexion à PostgreSQL
def connect_to_postgres():
    print("Connexion à PostgreSQL...")
    while True:
        try:
            conn = psycopg2.connect(
                host="postgres",
                database="DB_Mastodon",
                user="fadi",
                password="fadi"
            )
            print("Connexion à PostgreSQL réussie.")
            return conn
        except Exception as e:
            print(f"Erreur de connexion à PostgreSQL: {e}. Tentative de reconnexion dans 5 secondes...")
            time.sleep(5)

# Connexion à la base de données
conn = connect_to_postgres()
cursor = conn.cursor()

# Requête pour récupérer les données de la table Mostodon_BRONZ_bis
query = "SELECT * FROM Mostodon_BRONZ_bis;"

try:
    cursor.execute(query)
    rows = cursor.fetchall()  # Récupérer toutes les lignes
    print("Données de la table Mostodon_BRONZ_bis :")
    
    # Affichage des données
    for row in rows:
        print(row)  # Chaque ligne est affichée

except Exception as e:
    print(f"Erreur lors de la récupération des données : {e}")

finally:
    # Fermer le curseur et la connexion à PostgreSQL
    cursor.close()
    conn.close()
    print("Connexion à PostgreSQL fermée.")


Connexion à PostgreSQL...
Connexion à PostgreSQL réussie.
Données de la table Mostodon_BRONZ_bis :
(113238317448910493, 'tolle_et_lege', '中野善夫', '<p>明日の朝も４時半に起きて朝顔の受粉作業をしなくてはならないのだ。</p>', 0, 0, 0)
(113238317443894590, 'BBCNews', 'BBC News (UK) :press:', '<p>Alcaraz beats Sinner in dramatic China Open final</p><p>Carlos Alcaraz comes from a set down to edge out world number one Jannik Sinner in a dramatic China Open final. <a href="https://press.coop/tags/press" class="mention hashtag" rel="nofollow noopener noreferrer" target="_blank">#<span>press</span></a></p><p><a href="https://www.bbc.com/sport/tennis/articles/ce8v6560lnro?utm_source=press.coop" rel="nofollow noopener noreferrer" target="_blank"><span class="invisible">https://www.</span><span class="ellipsis">bbc.com/sport/tennis/articles/</span><span class="invisible">ce8v6560lnro?utm_source=press.coop</span></a></p>', 0, 0, 0)
(113238317437225454, 'B_Whitewind', 'Bishop Whitewind', '<p>Now that I don\'t pay bills feel free to co